# Drone Visual Navigation Project

## Backend Structure Overview

---

## PART 1: Environment Representation

**What it is:** The world where the drone operates

**What you need to code:**

- Grid/Map system (2D or 3D coordinates)
- Obstacles placement
- Goal/destination location
- Drone starting position

---

## PART 2: Drone Model

**What it is:** The drone's properties and capabilities

**What you need to code:**

- Current position
- Battery level
- Movement capabilities (speed, range)
- Sensor range (how far it can "see")

---

## PART 3: Game Theory Components

### Players

- **Drone** (decision-maker)
- **Environment** (nature/adversary)

### Strategies/Actions

#### DRONE STRATEGIES

**Pure strategies:**

```
{move_up, move_down, move_left, move_right, stay, rotate}
```

**Mixed strategy (σ_drone):**
Probability distribution over pure strategies

_Example:_

```
σ = (0.4·move_forward, 0.3·move_left, 0.2·move_right, 0.1·stay)
```

#### ENVIRONMENT STRATEGIES

**Pure strategies:**

```
{clear_path, obstacle_ahead, low_visibility, sensor_noise, lighting_change}
```

**Mixed strategy (σ_env):**
Probability distribution representing environmental uncertainty

_Example:_

```
σ = (0.5·clear, 0.3·obstacle, 0.2·low_vis)
```

### Payoff Function

```
u_drone(s_drone, s_env) = w1·mission_success
                        - w2·energy_consumed
                        - w3·collision_risk
                        + w4·map_quality
```

**Constraint:** `w1 + w2 + w3 + w4 = 1` (weights sum to 1)

#### Detailed Component Calculations

**mission_success:**

- `1.0` if reached goal
- `distance_to_goal / initial_distance` (partial credit)
- `0` if collision

**energy_consumed:**

- `battery_used / total_battery`
- Each move costs energy

**collision_risk:**

- `distance_to_nearest_obstacle^(-1)`
- Higher when close to obstacles

**map_quality:**

- `unexplored_area / total_area`
- Rewards exploration

#### Example Payoff Matrix (2×2)

|             | **Clear** | **Obstacle** |
| ----------- | --------- | ------------ |
| **Forward** | (10, -5)  | (-20, 5)     |
| **Stay**    | (2, 0)    | (2, 0)       |

Format: `(drone_payoff, env_payoff)`

---

## PART 4: Decision-Making Algorithms

### Algorithm 1: Minimax (Adversarial Environment)

**Assumptions:**

1. Adversarial environment (worst-case scenario)
2. Drone minimizes the maximum loss the environment can cause
3. Related to dominant strategies

**Mathematical formulation:**

```
s*_drone = arg max    min    u_drone(s_drone, s_env)
           s_drone  s_env ∈ S_env
```

**Translation:** "Choose the action that gives the best payoff even if environment picks the worst response"

### Algorithm 2: Nash Equilibrium

For finding stable strategies

### Algorithm 3: Bayesian Game

For uncertainty handling and incomplete information

---

## HYBRID STRATEGY

### Phase 1: Initial Navigation (high uncertainty)

- Use **BAYESIAN GAME** with cautious priors
- Gather sensor information

### Phase 2: Known Dangerous Areas

- Switch to **MINIMAX** (adversarial)
- Guarantee safety

### Phase 3: Open Areas

- Use **NASH EQUILIBRIUM** (mixed)
- Balance speed vs. safety optimally

### Phase 4: Near Goal

- Pure strategy (dominant)
- Direct path if clear

---

## PART 5: Simulation Engine

Main simulation loop implementation

---

## PART 6: Performance Metrics

**What it is:** Measuring how well your solution works

### Metrics Available

#### Pareto Optimality

A solution is Pareto optimal if no other solution improves one objective without worsening another

_Check:_ Can we reach goal faster WITHOUT using more battery?

#### Security Level

```
min_security_level = min    u_drone(s_drone, s_env)
                     s_env
```

Worst-case guaranteed payoff

#### Nash Equilibrium Quality

```
Efficiency = u(Nash) / u(Optimal)
```

Compare: Payoff at Nash vs. Optimal possible payoff

#### Basic Metrics

- Success rate (did it reach goal?)
- Path efficiency (shortest path vs actual path)
- Battery usage
- Collision count
- Time taken

**Example metrics object:**

```python
metrics = {
    'success': True/False,
    'path_length': 15,
    'battery_used': 45,
    'collisions': 0,
    'computation_time': 2.5  # seconds
}
```

---

## PART 7: Comparison

Compare approaches with and without game theory:

```python
results_with_game_theory = run_simulation(use_minimax=True)
results_without = run_simulation(use_minimax=False)
```

---

## Important Remarks

### Algorithm Selection Guide

| Algorithm            | Environment Type   | Use Case                                                                    |
| -------------------- | ------------------ | --------------------------------------------------------------------------- |
| **MINIMAX**          | Adversarial        | Worst-case thinking, environment actively opposes drone                     |
| **NASH EQUILIBRIUM** | Neutral/Stochastic | Realistic scenarios, environment doesn't "try" to hurt drone                |
| **BAYESIAN**         | Unknown type       | Incomplete information, drone doesn't know true environment state initially |

---

## Project File Structure

```
backend/
│
├── core/
│   ├── environment.py      # PART 1: Grid, obstacles
│   ├── drone.py            # PART 2: Drone model
│   └── sensor.py           # Drone vision simulation
│
├── game_theory/
│   ├── payoff.py           # PART 3C: Payoff function
│   ├── minimax.py          # PART 4: Minimax algorithm
│   ├── nash.py             # PART 4: Nash equilibrium
│   └── strategies.py       # PART 3B: All strategies
│
├── simulation/
│   ├── engine.py           # PART 5: Main simulation loop
│   ├── metrics.py          # PART 6: Performance tracking
│   └── logger.py           # Data logging
│
├── evaluation/
│   └── compare.py          # PART 7: Compare approaches
│
└── main.py                 # Run everything
```

# Drone Visual Navigation Project - File Descriptions

## PERSON 1 FILES

### `backend/core/environment.py`

This file contains the **Environment** class that represents the world where the drone operates.

**What it contains:**

- A grid system (like a chessboard) with specific width and height
- A list of all obstacles placed on the grid (walls, buildings, etc.)
- The goal position - where the drone needs to reach
- The starting position - where the drone begins
- Methods to check if a position is valid (not out of bounds, not hitting an obstacle)
- A function to calculate distance from any position to the goal
- A function to find the nearest obstacle from any given position
- A method to get the current state of the environment (snapshot of everything)

**Example:** If you have a 20x20 grid, this file lets you say "put an obstacle at position (5,5)" and "the goal is at (18,18)". It also answers questions like "Is position (10,10) safe to move to?" or "How far is the drone from the goal?"

---

### `backend/game_theory/strategies.py`

This file defines all possible actions (strategies) for both the drone and the environment.

**What it contains:**

**For the Drone:**

- A list of all pure strategies (single actions): move up, move down, move left, move right, stay in place, rotate
- A method to create mixed strategies (probability distributions over actions). For example: "40% chance move forward, 30% move left, 20% move right, 10% stay"
- A function to randomly sample one action from a mixed strategy based on the probabilities

**For the Environment:**

- A list of environmental conditions that can occur: clear path, obstacle suddenly appears, low visibility (fog/darkness), sensor noise (interference), lighting changes
- Similar methods to create mixed strategies for the environment (probability distributions of conditions)
- A function to sample which environmental condition actually occurs

**Example:** The drone might have a cautious strategy like "60% stay, 40% move forward" when uncertain. The environment might have "70% clear path, 20% obstacle, 10% low visibility" to represent typical conditions.

---

### `backend/game_theory/payoff.py`

This file contains the **PayoffFunction** class that calculates how good or bad each decision is.

**What it contains:**

- Four weight parameters (w1, w2, w3, w4) that must sum to 1.0, representing the importance of each component
- A function to calculate **mission success**: Returns 1.0 if the goal is reached, a partial score based on how close you got, or 0 if there was a collision
- A function to calculate **energy consumed**: Returns the fraction of battery used (0 to 1)
- A function to calculate **collision risk**: Returns a danger score based on how close the drone is to obstacles (closer = higher risk)
- A function to calculate **map quality**: Returns an exploration score - how much of the map has been explored
- The main **compute_payoff** function that combines all four components using the weights to give a single number representing "how good is this situation"
- A method to generate a complete payoff matrix showing payoffs for every possible combination of drone action and environment condition

**Example:** If the drone moves forward but there's an obstacle ahead, the payoff might be: (0.4 × 0) - (0.2 × 0.1) - (0.3 × 0.8) + (0.1 × 0.2) = -0.22 (negative means bad decision). If it stays safe and makes progress, the payoff might be +0.6 (positive means good decision).

---

### `backend/simulation/logger.py` (Optional Helper)

This file handles recording and saving simulation data.

**What it contains:**

- A logger class that keeps track of what happens during the simulation
- A method to log each step: what action was taken, what the payoff was, the drone's position, battery level, etc.
- A method to save all logged data to a file (CSV or JSON format) for later analysis
- Storage of the complete history of the simulation

**Example:** After running a simulation, you can review the log to see: "At step 15, the drone was at position (7,8), chose to move right, had 67% battery remaining, and received a payoff of 0.45."

---

## PERSON 2 FILES

### `backend/core/drone.py`

This file contains the **Drone** class that represents the drone itself with all its properties.

**What it contains:**

- The drone's current position on the grid (x, y coordinates)
- Battery level (current charge and maximum capacity)
- The complete path history - every position the drone has visited
- Energy cost per movement (how much battery each action consumes)
- A **move** function that executes an action: updates the drone's position based on the action (if moving up, y increases by 1), consumes battery, and adds the new position to the path history
- Functions to get current position and battery level
- A function to check if the drone is still "alive" (has battery remaining)
- A function that returns all valid actions from the current position (can't move up if there's a wall there)

**Example:** A drone starts at (1,1) with 100% battery. After moving right 3 times, it's now at (4,1) with 97% battery, and its path history is [(1,1), (2,1), (3,1), (4,1)].

---

### `backend/game_theory/minimax.py`

This file contains the **MinimaxSolver** class that implements the minimax algorithm for adversarial environments.

**What it contains:**

- The main **minimax_decision** function that implements the core algorithm logic:
  - For each possible drone action, it calculates the worst-case scenario (what's the minimum payoff if the environment picks its worst response?)
  - Then it chooses the action that has the best worst-case outcome (maximize the minimum)
- An **evaluate_action** function that, given one specific drone action, checks all possible environment responses and finds which response is worst for the drone
- A **get_worst_case_payoff** function that calculates the guaranteed minimum payoff for an action
- A **solve** wrapper function that orchestrates everything and returns the optimal action

**Philosophy:** "Assume the environment is trying to hurt you. What's the safest action that guarantees the best outcome even in the worst case?"

**Example:** If moving forward could give +10 payoff (if clear) or -20 payoff (if obstacle), and staying gives +2 in all cases, minimax chooses "stay" because its worst case (+2) is better than forward's worst case (-20).

---

## PERSON 3 FILES

### `backend/core/sensor.py`

This file contains the **DroneSensor** class that simulates the drone's vision and sensing capabilities.

**What it contains:**

- A detection range parameter (how many grid cells away the drone can see)
- A visibility level (0 to 1, where 1 is perfect visibility, 0.5 is foggy, etc.)
- A **scan_environment** function that returns all obstacles the drone can currently see within its range
- A **detect_obstacles** function that checks specific directions (up, down, left, right) and reports how far away obstacles are in each direction, or None if no obstacle in that direction
- An **update_visibility** function that adjusts visibility based on environmental conditions (fog reduces visibility)
- A **get_observable_region** function that returns all grid coordinates the drone can currently observe

**Example:** A drone at position (10,10) with range 5 can see positions from (5,5) to (15,15). If there's an obstacle at (10,13), the sensor returns "obstacle 3 cells ahead". If fog occurs, visibility drops to 0.6, reducing effective range.

---

### `backend/game_theory/nash.py`

This file contains the **NashEquilibriumSolver** class that finds stable strategy pairs.

**What it contains:**

- The main **find_nash_equilibrium** function that solves for Nash equilibrium given a payoff matrix. It uses mathematical optimization techniques to find probability distributions where neither player wants to change their strategy
- A **support_enumeration** alternative method that tries different combinations of strategies to find equilibrium
- A **best_response_drone** function: given what the environment is doing (its mixed strategy), what's the best thing the drone can do?
- A **best_response_env** function: given what the drone is doing, what's the best environment response?
- An **is_nash_equilibrium** checker that verifies if a strategy pair is actually a Nash equilibrium (neither player benefits from changing)
- A **solve** function that computes the Nash equilibrium and samples an action from the resulting mixed strategy

**Philosophy:** "Find a stable balance where the drone's strategy is optimal given the environment's strategy, and vice versa. Nobody wants to deviate."

**Example:** Nash might find: Drone should use (50% forward, 30% stay, 20% turn) and Environment naturally produces (80% clear, 15% obstacle, 5% fog). Neither can improve by changing unilaterally.

---

## PERSON 4 FILES

### `backend/game_theory/bayesian.py`

This file contains the **BayesianGameSolver** class that handles uncertainty and learning.

**What it contains:**

- A belief state storage - the drone's current beliefs about what type of environment it's in (is it adversarial? neutral? helpful?)
- An **initialize_beliefs** function that sets starting beliefs before the drone has any information. Example: "I think there's 30% chance the environment is adversarial, 50% neutral, 20% helpful"
- An **update_beliefs** function that uses Bayes' theorem to update beliefs after observing what actually happened. If an obstacle appeared when the drone moved forward, this increases belief that the environment is adversarial
- An **expected_utility** function that calculates the expected payoff of an action by averaging over all possible environment types, weighted by the belief probabilities
- A **bayesian_decision** function that chooses the action with the highest expected utility based on current beliefs
- A **solve** wrapper function

**Philosophy:** "I don't know what kind of environment this is, but I'll maintain beliefs and update them as I learn. I'll choose actions based on expected value over my uncertain beliefs."

**Example:** Initially believe 50-50 adversarial vs neutral. After 5 safe moves, update to 20% adversarial, 80% neutral. Now take bolder actions because you've learned the environment is probably friendly.

---

### `backend/simulation/engine.py`

This file contains the **SimulationEngine** class that runs the actual simulation - the "brain" that coordinates everything.

**What it contains:**

- References to the environment, drone, and which algorithm to use (minimax, nash, or bayesian)
- A step counter and history log to track progress
- Instances of all three algorithm solvers (minimax, nash, bayesian)
- A **step** function that executes one time step of the simulation:
  1. Selects which algorithm to use
  2. Gets the recommended action from that algorithm
  3. Executes the action (moves the drone)
  4. Updates the environment state
  5. Logs what happened
  6. Checks if done (reached goal, crashed, or out of battery)
- A **select_action** function that routes to the appropriate algorithm based on the current mode
- A **run_simulation** function that runs the complete simulation from start to finish, up to a maximum number of steps, and returns the final metrics
- A **collect_metrics** function that gathers all results at the end
- A **reset** function to restart the simulation

**Example:** You tell the engine "use minimax algorithm". It runs step-by-step: gets minimax action, moves drone, checks status, logs data, repeats until goal reached or failure. Returns: "Reached goal in 47 steps, used 68% battery, 0 collisions."

---

### `backend/simulation/metrics.py`

This file contains the **PerformanceMetrics** class that measures and evaluates how well the drone performed.

**What it contains:**

- A **calculate_success_rate** function that checks: Did the drone reach the goal? Returns True/False
- A **calculate_path_efficiency** function that compares the actual path length to the optimal (shortest) path length. Returns a ratio (1.0 = perfect, 0.5 = took twice as long as needed)
- A **calculate_pareto_optimality** function that checks if the solution is Pareto optimal by comparing it to other solutions. Returns whether any solution exists that's better in one dimension without being worse in another
- A **calculate_security_level** function that computes the worst-case guaranteed payoff - the minimum the drone can expect no matter what happens
- A **calculate_nash_efficiency** function that compares the payoff at Nash equilibrium to the theoretically optimal payoff (price of anarchy)
- A **generate_report** function that combines all metrics into a comprehensive dictionary with success status, path length, battery used, number of collisions, computation time, whether Pareto optimal, security level, and Nash efficiency

**Example:** Final report might say: "Success: Yes, Path length: 52 (optimal was 45, efficiency: 86.5%), Battery used: 71%, Collisions: 0, Pareto optimal: True, Security level: 0.42, Nash efficiency: 0.89"

---

### `backend/evaluation/compare.py`

This file contains the **AlgorithmComparator** class that compares all three algorithms against each other.

**What it contains:**

- Storage for the environment and drone to test with
- A results dictionary to store outcomes from each algorithm
- A **run_all_algorithms** function that:
  - Runs minimax multiple times (e.g., 10 trials)
  - Runs nash multiple times
  - Runs bayesian multiple times
  - Stores all results for statistical comparison
- A **compare_metrics** function that performs statistical analysis on the results: calculates mean, standard deviation, min, max for each metric across algorithms
- A **visualize_comparison** function (optional) that creates charts comparing the algorithms
- A **generate_comparison_report** function that produces a final report answering: "Which algorithm was best overall? Which was fastest? Which used least battery? Which was safest?"

**Example:** After running 10 trials of each, the report might say: "Minimax: 90% success rate, average 63 steps, 72% battery. Nash: 95% success, average 58 steps, 68% battery. Bayesian: 88% success, average 61 steps, 70% battery. Winner: Nash equilibrium."

---

### `backend/main.py`

This is the **entry point** - the file you actually run to start everything.

**What it contains:**

- The main function that orchestrates the entire project
- Setup code that:
  1. Creates the environment (20x20 grid, adds obstacles, sets goal and start positions)
  2. Creates the drone (sets starting position and battery)
  3. Runs a single algorithm test (e.g., test minimax alone first)
  4. Runs the comparison of all algorithms
  5. Calculates and displays final metrics
  6. Prints results to the console
- The `if __name__ == "__main__"` block that runs when you execute the file

**Example flow:** You run `python main.py` → It creates a 20x20 grid with obstacles, creates a drone at (1,1), tests minimax (prints "Minimax reached goal in 47 steps"), then runs all three algorithms 5 times each, and finally prints "Nash performed best with 95% success rate."

---

## Summary of How Files Work Together

1. **Person 1's files** create the foundation: the world (environment), the rules (strategies), and the scoring system (payoff)

2. **Person 2 and 3's files** build the intelligence: minimax makes safe decisions, nash finds balance, and sensors provide awareness

3. **Person 4's files** bring it all together: bayesian adds learning, the engine runs simulations, metrics measure performance, comparison evaluates algorithms, and main.py starts everything

**Analogy:** It's like building a video game: Person 1 makes the map and rules, Person 2 makes an AI that's very defensive, Person 3 makes an AI that's balanced, Person 4 makes an AI that learns, runs the game, keeps score, and determines who wins!